In [34]:
import pandas as pd
import json
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col
from snowflake.snowpark.types import StructType, StructField, StringType, FloatType, TimestampType, StructType, StructField, StringType, IntegerType, FloatType, DateType
import os

In [48]:
# Read the JSON file back into a dictionary
with open("connection_parameters.json", "r") as json_file:
    connection_parameters = json.load(json_file)

table_name = "players"

In [49]:
df = pd.read_csv(f'{table_name}.csv')
df

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,205.0,75.0,R,R,4/6/04,9/28/13,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,...,Aaron,Henry Louis,180.0,72.0,R,R,4/13/54,10/3/76,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,...,Aaron,Tommie Lee,190.0,75.0,R,R,4/10/62,9/26/71,aarot101,aaronto01
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190.0,75.0,R,R,7/26/77,10/3/90,aased001,aasedo01
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184.0,73.0,L,L,9/10/01,4/13/06,abada001,abadan01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18584,zupcibo01,1966.0,8.0,18.0,USA,PA,Pittsburgh,NaN,NaN,NaN,...,Zupcic,Robert,220.0,76.0,R,R,9/7/91,8/4/94,zupcb001,zupcibo01
18585,zupofr01,1939.0,8.0,29.0,USA,CA,San Francisco,2005.0,3.0,25.0,...,Zupo,Frank Joseph,182.0,71.0,L,R,7/1/57,5/9/61,zupof101,zupofr01
18586,zuvelpa01,1958.0,10.0,31.0,USA,CA,San Mateo,NaN,NaN,NaN,...,Zuvella,Paul,173.0,72.0,R,R,9/4/82,5/2/91,zuvep001,zuvelpa01
18587,zuverge01,1924.0,8.0,20.0,USA,MI,Holland,2014.0,9.0,8.0,...,Zuverink,George,195.0,76.0,R,R,4/21/51,6/15/59,zuveg101,zuverge01


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18589 entries, 0 to 18588
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   playerID      18589 non-null  object 
 1   birthYear     18444 non-null  float64
 2   birthMonth    18274 non-null  float64
 3   birthDay      18117 non-null  float64
 4   birthCountry  18516 non-null  object 
 5   birthState    17975 non-null  object 
 6   birthCity     18389 non-null  object 
 7   deathYear     9225 non-null   float64
 8   deathMonth    9224 non-null   float64
 9   deathDay      9223 non-null   float64
 10  deathCountry  9223 non-null   object 
 11  deathState    9169 non-null   object 
 12  deathCity     9219 non-null   object 
 13  nameFirst     18549 non-null  object 
 14  nameLast      18589 non-null  object 
 15  nameGiven     18549 non-null  object 
 16  weight        17717 non-null  float64
 17  height        17780 non-null  float64
 18  bats          17399 non-nu

In [38]:
df['order_date']

0         1/3/21
1         1/4/21
2         1/4/21
3         1/4/21
4         1/5/21
          ...   
8544    12/30/24
8545    12/30/24
8546    12/30/24
8547    12/30/24
8548    12/30/24
Name: order_date, Length: 8549, dtype: object

In [ ]:
# Assuming df is your DataFrame
#df['order_date'] = pd.to_datetime(df['order_date'], format='%m/%d/%y')

# If the year format is '21' for 2021 and '24' for 2024, the above code will work.
# However, if the year format is '2021' and '2024', you should use '%m/%d/%Y' instead.

# Check the result
#print(df['order_date'])

0      2021-01-03
1      2021-01-04
2      2021-01-04
3      2021-01-04
4      2021-01-05
          ...    
8544   2024-12-30
8545   2024-12-30
8546   2024-12-30
8547   2024-12-30
8548   2024-12-30
Name: order_date, Length: 8549, dtype: datetime64[ns]


In [40]:
session = Session.builder.configs(connection_parameters).create()

In [41]:
# Function to map Pandas dtypes to Snowpark types
def map_dtype_to_snowpark(dtype):
    if pd.api.types.is_string_dtype(dtype):
        return StringType()
    elif pd.api.types.is_integer_dtype(dtype):
        return IntegerType()
    elif pd.api.types.is_float_dtype(dtype):
        return FloatType()
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return DateType()
    else:
        raise ValueError(f"Unsupported data type: {dtype}")

# Create Snowpark schema dynamically
schema = StructType([
    StructField(name, map_dtype_to_snowpark(dtype)) for name, dtype in zip(df.columns, df.dtypes)
])

schema

StructType([StructField('TRANSACTION_ID', StringType(), nullable=True), StructField('CUSTOMER_ID', IntegerType(), nullable=True), StructField('ORDER_ID', StringType(), nullable=True), StructField('ORDER_DATE', DateType(), nullable=True), StructField('PRODUCT_ID', StringType(), nullable=True), StructField('UNITS', IntegerType(), nullable=True)])

In [42]:
snowpark_df = session.create_dataframe(df, schema=schema)

In [43]:
for column_name in snowpark_df.columns:
    snowpark_df = snowpark_df.with_column_renamed(column_name, column_name.upper())

In [44]:
snowpark_df.show()

-----------------------------------------------------------------------------------------------------------
|"TRANSACTION_ID"  |"CUSTOMER_ID"  |"ORDER_ID"            |"ORDER_DATE"         |"PRODUCT_ID"   |"UNITS"  |
-----------------------------------------------------------------------------------------------------------
|T0001             |21240          |US-2021-103800-44199  |2021-01-03 00:00:00  |CHO-MIL-31000  |2        |
|T0002             |18127          |US-2021-112326-44200  |2021-01-04 00:00:00  |CHO-TRI-54000  |2        |
|T0003             |18127          |US-2021-112326-44200  |2021-01-04 00:00:00  |CHO-NUT-13000  |3        |
|T0004             |18127          |US-2021-112326-44200  |2021-01-04 00:00:00  |CHO-SCR-58000  |3        |
|T0005             |25041          |US-2021-141817-44201  |2021-01-05 00:00:00  |CHO-TRI-54000  |3        |
|T0006             |28581          |US-2021-167199-44202  |2021-01-06 00:00:00  |CHO-SCR-58000  |13       |
|T0007             |28581   

In [46]:
snowpark_df.dtypes

[('TRANSACTION_ID', 'string(16777216)'),
 ('CUSTOMER_ID', 'bigint'),
 ('ORDER_ID', 'string(16777216)'),
 ('ORDER_DATE', 'timestamp'),
 ('PRODUCT_ID', 'string(16777216)'),
 ('UNITS', 'bigint')]

In [47]:
# Write the Snowpark DataFrame to the Snowflake table
snowpark_df.write.mode('overwrite').save_as_table(table_name)

In [33]:
session.close()